# Text cleaning

A notebook to better explain text cleaning process and help building Transformers and make pipelines.

This is a refactoring of part of the job made in [Rakuten_preprocessing.py](Rakuten_preprocessing.py) by Julien Fournier

In [13]:
import pandas as pd
import numpy as np
import os
import re

BASE_DIR = os.getcwd()
directory_name = os.path.basename(BASE_DIR)
if directory_name != "RakutenTeam": 
    BASE_DIR = os.path.realpath(os.path.join(os.path.dirname(__name__), '..'))
    os.chdir(BASE_DIR)
DATA_DIR = os.path.join(BASE_DIR, 'data')
print(DATA_DIR, BASE_DIR)

C:\Users\mangg\projects\RakutenTeam\data C:\Users\mangg\projects\RakutenTeam


In [14]:
from src.data_loader import import_data

folder_path = os.path.join(DATA_DIR, "raw")
print(folder_path)

C:\Users\mangg\projects\RakutenTeam\data\raw


In [15]:
df = import_data(folder_path=folder_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84916 entries, 0 to 84915
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prdtypecode  84916 non-null  int64 
 1   designation  84916 non-null  object
 2   description  55116 non-null  object
 3   productid    84916 non-null  int64 
 4   imageid      84916 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 5.9+ MB


On a des NaN pour description et pas pour designation (qui doit être un champ obligatoire).

In [16]:
df.head()

,prdtypecode,designation,description,productid,imageid
0,10,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046
1,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237
2,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978
3,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496
4,2705,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786


Nous avons 
Concentrons nous ici sur le texte.
Nous avons deux colonnes non pertinentes et deux à étudier: designation et description.

In [17]:
descriptions = df.description

designation = df.designation

Travaillons sur les descriptions, notamment celles qui ne sont pas NaN

In [18]:
filter = descriptions.notnull()
filtered_descriptions = descriptions[filter]

In [19]:
from sklearn import set_config
set_config(display="diagram")

In [20]:
from src.features.text.pipelines.cleaner import CleanTextPipeline


pipe = CleanTextPipeline()
pipe


CleanTextPipeline()

In [21]:
url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
filename_pattern = r'\b(?<!\d\.)\w+\.(txt|jpg|png|docx|pdf)\b'
bad_html_pattern = r'nbsp|&amp|& nbsp|'
space_around_pattern = r'(\d+|[-.,!¡;；:¯…„“\§«»—°•£❤☆(){}\[\]"@#$%^&*+=|<>~`‘’¬])'
space_before_pattern = r'(?<=[a-zÀ-ÿ]|[.,!;:\§«»°])([A-Z])(?=[a-zÀ-ÿ])'

In [22]:
new_descriptions = pipe.fit_transform(descriptions)


C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:308: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:308: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:331: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')
C:\Users\mangg\projects\RakutenTeam\src\features\text\transformers\cleaners.py:331: MarkupResemblesLocator

In [23]:
descriptions.str.count(url_pattern).sum(), descriptions.str.count(filename_pattern).sum(), descriptions.str.count(bad_html_pattern).sum(), descriptions.str.count(space_around_pattern).sum(), descriptions.str.count(space_before_pattern).sum()


(233.0, 98.0, 9877.0, 3154773.0, 88066.0)

In [24]:
new_descriptions.str.count(url_pattern).sum(), new_descriptions.str.count(filename_pattern).sum(), new_descriptions.str.count(bad_html_pattern).sum(), new_descriptions.str.count(space_around_pattern).sum(), new_descriptions.str.count(space_before_pattern).sum()


(0.0, 0.0, 2404.0, 1572265.0, 84123.0)